# Prepare the environment and data

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .appName("Final Project") \
     .getOrCreate()
sc = spark.sparkContext      # get the context
sc

<SparkContext master=yarn appName=Final Project>

In [24]:
import pandas as pd

In [4]:
features = spark.read.format("csv").option("header", "true").load("s3://anly502final/fma_dataset/fma_dataset.csv")

In [7]:
tracks = spark.read.format("csv").option("header", "true").load("s3://anly502final/fma_dataset/tracks.csv")

In [8]:
#Combine two csv file by tracks_id column
Join_table = features.join(tracks["track_id","type"], features.feature == tracks.track_id)

In [9]:
#convert the type of track_id column into integer
from pyspark.sql.types import *
Join_table = Join_table.withColumn("track_id", Join_table["track_id"].cast(IntegerType()))

In [10]:
#order by track_id column
from pyspark.sql.functions import col
Join_table = Join_table.orderBy(Join_table.track_id)

In [11]:
#delete duplicate column
Join_table = Join_table.drop('feature')

### .printSchema()

There are 520 columns in the following dataframe. 
The first 518 columns are features extracted by LibROSA, which is a python package for music and audio analysis. 
The last two columns are respectively the identity column "track_id" and the target variable "type".

In [12]:
Join_table.printSchema()

root
 |-- chroma_cens1: string (nullable = true)
 |-- chroma_cens2: string (nullable = true)
 |-- chroma_cens3: string (nullable = true)
 |-- chroma_cens4: string (nullable = true)
 |-- chroma_cens5: string (nullable = true)
 |-- chroma_cens6: string (nullable = true)
 |-- chroma_cens7: string (nullable = true)
 |-- chroma_cens8: string (nullable = true)
 |-- chroma_cens9: string (nullable = true)
 |-- chroma_cens10: string (nullable = true)
 |-- chroma_cens11: string (nullable = true)
 |-- chroma_cens12: string (nullable = true)
 |-- chroma_cens13: string (nullable = true)
 |-- chroma_cens14: string (nullable = true)
 |-- chroma_cens15: string (nullable = true)
 |-- chroma_cens16: string (nullable = true)
 |-- chroma_cens17: string (nullable = true)
 |-- chroma_cens18: string (nullable = true)
 |-- chroma_cens19: string (nullable = true)
 |-- chroma_cens20: string (nullable = true)
 |-- chroma_cens21: string (nullable = true)
 |-- chroma_cens22: string (nullable = true)
 |-- chroma_ce

Create a TempView and filter out distinct values of target variables:

Now we have four categories: album, live performance, single tracks and radio program.

In [13]:
Join_table.createOrReplaceTempView("Join_table")

In [14]:
cleaned_df = spark.sql("select * from Join_table where type in ('Album', 'Live Performance', 'Single Tracks', 'Radio Program') ")

### Exploratory data analysis

### Modelling

In [37]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model
from pyspark.ml.feature import StringIndexer

In [ ]:
df = cleaned_df.toPandas()
new_df = pd.DataFrame(df['chroma_cens1'])
new_df['chroma_cqt85'] = df['chroma_cqt85']
new_df['chroma_stft169'] = df['chroma_stft169']
new_df['mfcc253'] = df['mfcc253']
new_df['spectral_bandwidth400'] = df['spectral_bandwidth400']
new_df['spectral_centroid407'] = df['spectral_centroid407']
new_df['spectral_contrast414'] = df['spectral_contrast414']
new_df['spectral_rolloff463'] = df['spectral_rolloff463']
new_df['tonnetz470'] = df['tonnetz470']
new_df['zcr512'] = df['zcr512']
new_df['type'] = df['type']

In [33]:
new_df['chroma_cens1'] = pd.to_numeric(new_df['chroma_cens1'], errors='coerce')
new_df['chroma_cqt85'] = pd.to_numeric(new_df['chroma_cqt85'], errors='coerce')
new_df['chroma_stft169'] = pd.to_numeric(new_df['chroma_stft169'], errors='coerce')
new_df['mfcc253'] = pd.to_numeric(new_df['mfcc253'], errors='coerce')
new_df['spectral_bandwidth400'] = pd.to_numeric(new_df['spectral_bandwidth400'], errors='coerce')
new_df['spectral_centroid407'] = pd.to_numeric(new_df['spectral_centroid407'], errors='coerce')
new_df['spectral_contrast414'] = pd.to_numeric(new_df['spectral_contrast414'], errors='coerce')
new_df['spectral_rolloff463'] = pd.to_numeric(new_df['spectral_rolloff463'], errors='coerce')
new_df['tonnetz470'] = pd.to_numeric(new_df['tonnetz470'], errors='coerce')
new_df['zcr512'] = pd.to_numeric(new_df['zcr512'], errors='coerce')


Append a new Column called label which maps the types Album, Live Performance, Single Tracks, Radio Program to 1, 2, 3, 4 respectively. 

In [39]:
def convert(line):
    try:
        if line == 'Album':
            return 1
        elif line == 'Live Performance':
            return 2
        elif line == 'Single Tracks':
            return 3
        elif line == 'Radio Program':
            return 4
    except:
        return 0
    

In [41]:
new_df['label'] = new_df['type'].apply(convert)
